You first need to install geemap to run this script. See https://geemap.org/installation/. Also this sample code is https://geemap.org/workshops/Crop_Mapping_2022/#esri-global-land-cover.

Load libraries

In [1]:
import geemap
import ee
import os
import getpass
ee.Authenticate()
ee.Initialize()

Set the path to specifiy where outputs will be saved. 

In [106]:
username = getpass.getuser()
if username == "WB495141":
    out_dir = 'C:/Users/WB495141/GitHub/tanzania_spatial_training_2024/data-raw/ndvi/'

out_dir = 'C:/Users/WB495141/GitHub/tanzania_spatial_training_2024/data-raw/esri_lc/'

Define the extent of region for which analysis will be performed

In [135]:
region = ee.Geometry.BBox(29.60168, -11.76347, 40.44495, -0.9857875)

Read ADM1 shapefile

In [136]:
poly = ee.FeatureCollection('projects/ee-tmasaki040685/assets/tza_admbnda_adm1_20181019')

Get ESRI landcover data

In [137]:
esri = ee.ImageCollection(
    "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS")
##.filterDate() specifies date
##.mosaic() turn it into image
##.clip() masks image to the boundary of poly


Get year-specific raster

In [138]:
esri_2017 = esri.filterDate("2017-01-01", "2017-12-31").filterBounds(region).mosaic() #extract ESRI landcover for 2017
esri_2021 = esri.filterDate("2021-01-01", "2021-12-31").filterBounds(region).mosaic() #extract ESRI landcover for 2021

Set legend for visualization

In [150]:
#assign colors to values
esri_vis = {
    'min': 1,
    'max': 11,
    'palette': [
        "#1A5BAB",
        "#358221",
        "#000000",
        "#87D19E",
        "#FFDB5C",
        "#000000",
        "#ED022A",
        "#EDE9E4",
        "#F2FAFF",
        "#C8C8C8",
        "#C6AD8D"
    ],
}

#assign legend to values
legend_dict = {
    '1 Water': '#1A5BAB',
    '2 Trees': '#358221',
    '4 Flooded Vegetation': '#87D19E5',
    '5 Crops': '#FFDB5C',
    '7 Built area': '#ED022A',
    '8 Bare ground': '#EDE9E4',
    '9 Snow/ice': '#F2FAFF',
    '10 Clouds': '#C8C8C8',
    '11 Rangeland': '#C6AD8D',
}

Let's map landcover for 2017

In [140]:
Map = geemap.Map()
Map.addLayer(esri_2017, esri_vis, "ESRI LULC 2017")
Map.add_legend(legend_title="ESRI Land Cover Classification", legend_dict=legend_dict)
Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Let's analyze cropland gain and loss. 

Note that eq() checks for equality, and in this case, it is checking if the values in esri_2017 are equal to 5. selfMask() creates a mask based on the condition specified in eq(5). It is creating a binary mask where pixels matching the condition (in this case, equal to 5) are set to 1, and all others are set to NA.

In [141]:
cropland_2017 = esri_2017.eq(5).selfMask()
cropland_2021 = esri_2021.eq(5).selfMask()

cropland_gain = esri_2017.neq(5).And(esri_2021.eq(5)).selfMask()
cropland_loss = esri_2017.eq(5).And(esri_2021.neq(5)).selfMask()

Map = geemap.Map()
Map.addLayer(cropland_2021, {"palette": "green"}, "Cropland 2021")
Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [142]:
Map = geemap.Map()
Map.addLayer(cropland_gain, {"palette": "yellow"}, "Cropland gain")
Map.addLayer(cropland_loss, {"palette": "red"}, "Cropland loss")
Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Compute % of cropland by using geemap.zonal_stats()

Set the filename for zonal statistics

In [143]:
filename = os.path.join(out_dir, "esri_cropland_share.csv")

In [144]:
cropland_2021_bin = cropland_2017.unmask(0) #replace missing values as 0
geemap.zonal_stats(
    cropland_2021_bin,
    poly,
    filename,
    stat_type="MEAN",
    scale=100, ##though the original resolution of the ESRI landcover data is at 10m, you will suffer from time out because it's computationally too heavy 
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\WB495141\GitHub\tanzania_spatial_training_2024\data-raw\esri_lc\esri_cropland_share.csv


Let's now test with forest coverage.

In [145]:
forest_2017 = esri_2017.eq(2).selfMask()
forest_2021 = esri_2021.eq(2).selfMask()

In [146]:
forest_gain = esri_2017.neq(2).And(esri_2021.eq(2)).selfMask()
forest_loss = esri_2017.eq(2).And(esri_2021.neq(2)).selfMask()

Map = geemap.Map()
Map.addLayer(forest_2017, {"palette": "brown"}, "Forest 2017", False)
Map.addLayer(forest_2021, {"palette": "cyan"}, "Forest 2021", False)

Map.addLayer(cropland_gain, {"palette": "yellow"}, "Forest gain")
Map.addLayer(cropland_loss, {"palette": "red"}, "Forest loss")
Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Now work with Hansen landcover data. See more on https://geemap.org/workshops/GEE_Workshop_2022_Part2/#forest-cover-change-analysis. 

In [147]:
dataset = ee.Image("UMD/hansen/global_forest_change_2022_v1_10").clip(region)

Get bandnames that are available in Hansen dataset

In [149]:
dataset.bandNames().getInfo()

['treecover2000',
 'loss',
 'gain',
 'lossyear',
 'first_b30',
 'first_b40',
 'first_b50',
 'first_b70',
 'last_b30',
 'last_b40',
 'last_b50',
 'last_b70',
 'datamask']

In [125]:
#get tree cover layer from hansen dataset
treecover = dataset.select(["treecover2000"])

#assign colors to values
treeCoverVisParam = {"min": 0, "max": 100, "palette": ["black", "green"]}

#plot
name1 = "Tree cover (%)"
Map = geemap.Map()
Map.addLayer(treecover, treeCoverVisParam, name1)
Map.add_colorbar(treeCoverVisParam, label=name1, layer_name=name1)
Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [126]:
treeloss = dataset.select(["loss"])
threshold = 1
Map = geemap.Map()
treeloss = treeloss.gte(threshold).selfMask()
Map.addLayer(treeloss, {"palette": "red"}, "Forest loss")
Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [131]:
filename = os.path.join(out_dir, "hansen_treeloss_share.csv")
treeloss_bin = treeloss.unmask(0)
geemap.zonal_stats(
    treeloss_bin,
    poly,
    filename,
    stat_type="MEAN",
    scale=30,
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\WB495141\GitHub\tanzania_spatial_training_2024\data-raw\esri_lc\hansen_treeloss_share.csv
